In [1]:
import requests
import pandas as pd
import numpy as np
import time
import random
from bs4 import BeautifulSoup
import aiohttp
import asyncio
import nest_asyncio
nest_asyncio.apply()
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv(r'C:\Users\9Suser\Desktop\模型訓練資料\tbrain_train_final_0603.csv')[['hyperlink', 'content']]

In [3]:
def number_p(soup, firstwords, endwords, link):    
    flag = False
    content = []
    
    
    if('mops' in link):
        tags = soup.find_all('font', class_=None, text=True, attrs={'data-reactid':None})
    elif('domestic.judicial' in link):   #司法院公告
        tags = soup.find_all('pre', class_=None, text=True, attrs={'data-reactid':None})
    else:
        tags = soup.find_all('p', class_=None, text=True, attrs={'data-reactid':None})
    
    
    
    
    if firstwords not in str(tags):
        for tag in tags:
            content.append(tag.contents[0])
            if endwords in tag.contents[0]:
                break
    else:
        for tag in tags:

            if firstwords in tag.contents[0]:
                flag = True

            if flag:
                content.append(tag.contents[0])
                if endwords in tag.contents[0]:
                    break
    
    return content

In [4]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}

In [5]:
async def crawl(session, index, single_data):
    print(index, single_data['hyperlink'])
    if('tvbs' not in single_data['hyperlink']):
        #try:
        if(('mingpao' in single_data['hyperlink']) & ('coolloud' in single_data['hyperlink'])):
            response = await session.get(single_data['hyperlink'])    
            response.encoding = 'utf-8'
            content = await response.text()
            content = content.text

        else:  
            response = await session.get(single_data['hyperlink'])
            content = await response.text()
        
        soup = BeautifulSoup(content.replace('<br>', '/n'), 'html.parser')

        firstwords = single_data['content'].split('###')[0][-6:-1]
        endwords = single_data['content'].split('###')[-1][1:6]
        all_content = number_p(soup, firstwords=firstwords, endwords=endwords, link=single_data['hyperlink'])
        single_data['content'] = all_content
        #except:
        #    single_data['content'] = ''
        #    print(single_data['hyperlink'], 'XXX')

    else:
        single_data['content'] = ''
        print(single_data['hyperlink'], 'XXX')

    await asyncio.sleep(0.1)

In [6]:
async def main(loop):
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
    async with aiohttp.ClientSession(loop=loop, headers=headers) as session:
        tasks = [crawl(session, index, single_data) for index, single_data in data.iterrows()]
        finished, unfinished = await asyncio.wait(tasks)  
        #all_results = [r.result() for r in finished]    # 獲取所有結果
        #return all_results

In [ ]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
loop = asyncio.get_event_loop()

In [ ]:
session = aiohttp.ClientSession(loop=loop, headers=headers)

In [ ]:
response = await session.get('http://domestic.judicial.gov.tw/abbs/wkw/WHD9HN03.jsp?crtid=PTD&filenm=ECE5DAECAF563DA171E34B03FC5185742FC90F879191D5DF')
content = await response.text()
soup = BeautifulSoup(content.replace('<br>', '/n'), 'html.parser')
tags = soup.find_all('pre', class_=None, text=True, attrs={'data-reactid':None})

In [ ]:
tags

In [ ]:
firstwords = data.loc[0,'content'].split('###')[0][-6:-1]
endwords = data.loc[0,'content'].split('###')[-1][1:6]

In [ ]:
all_content = number_p(soup, firstwords=firstwords, endwords=endwords)

In [ ]:
all_content

In [7]:
loop = asyncio.get_event_loop()
loop.run_until_complete(main(loop))

488 https://news.tvbs.com.tw/politics/1203607
https://news.tvbs.com.tw/politics/1203607 XXX
580 https://www.mirrormedia.mg/story/20191204ent009/
2190 https://udn.com/news/story/7253/3929513
2282 https://m.ctee.com.tw/livenews/aj/chinatimes/20190919003025-260410
2374 https://news.cnyes.com/news/id/4359691
4584 https://m.ctee.com.tw/livenews/aj/ctee/a78817002019120918153983
489 https://news.mingpao.com/ins/%e6%b8%af%e8%81%9e/article/20191019/s00001/1571456500906/%e3%80%90%e9%80%83%e7%8a%af%e6%a2%9d%e4%be%8b%e3%80%91%e6%9e%97%e9%84%ad-%e8%ad%a6%e4%b8%8d%e6%9c%83%e3%80%8c%e7%89%b9%e7%99%bb%e3%80%8d%e7%94%a8%e6%9a%b4%e5%8a%9b-%e6%94%bf%e5%ba%9c%e4%b8%8d%e5%ae%b9%e5%bf%8d%e9%81%95%e6%b3%95%e6%9a%b4%e5%8a%9b%e8%a1%8c%e7%82%ba%e5%8c%85%e6%8b%ac%e8%ad%a6%e5%af%9f
581 https://udn.com/news/story/11315/4113163
2191 https://udn.com/news/story/6809/4138584
2283 https://www.mirrormedia.mg/story/20191127ent026/
2375 https://www.mirrormedia.mg/story/20191227ent008/
490 https://news.cnyes.com/news/id/43

3819 https://www.hk01.com/社會新聞/382279/禁蒙面法-直播-沙田多處遭快閃縱火-防暴警放催淚彈
3911 https://www.mirrormedia.mg/story/20191014soc006/
2026 https://www.businesstoday.com.tw/article/category/80392/post/201908020006/
2118 https://www.bnext.com.tw/article/55868/the-goldman-sachs-group-ceo-david-solomon
3728 https://www.hk01.com/社會新聞/386149/建造業開工不足15工種全面凍薪-工會籲勿拉布促撥款
3820 https://www.mirrormedia.mg/story/20191213bus001/
3912 https://www.chinatimes.com/newspapers/20191220001160-260203
2027 https://news.cnyes.com/news/id/4405668
2119 https://news.mingpao.com/ins/%e6%b8%af%e8%81%9e/article/20191005/s00001/1570257763551/%e3%80%90%e7%a6%81%e8%92%99%e9%9d%a2%e6%b3%95-%e7%9f%ad%e7%89%87%e3%80%91%e6%9e%97%e9%84%ad%e9%8c%84%e5%bd%b1%e8%ac%9b%e8%a9%b1-%e6%b8%af%e6%98%a8%e5%ba%a6%e9%81%8e%e9%bb%91%e6%9a%97%e4%b8%80%e5%a4%9c-%e5%85%ac%e5%85%b1%e5%ae%89%e5%85%a8%e5%8f%97%e5%8d%b1%e5%ae%b3%e6%98%af%e7%ab%8b%e6%b3%95%e7%90%86%e6%93%9a
3729 https://www.bnext.com.tw/article/54580/new-wave-of-marketing-martech
3821 https://w

1921 https://news.cnyes.com/news/id/4358951
3439 https://udn.com/news/story/7321/4066945
3531 http://www.coolloud.org.tw/node/93392
220 https://m.ctee.com.tw/livenews/ch/chinatimes/20190904003871-260405
1830 http://technews.tw/2019/12/23/enisa-good-practices-for-security-of-iot/
1922 https://udn.com/news/story/120538/3987038
3440 http://finance.technews.tw/2019/12/22/over-97percent-of-listed-counter-companies-lack-visibility/
3532 https://m.ctee.com.tw/livenews/ch/chinatimes/20191031002150-260405
221 https://news.tvbs.com.tw/politics/1200870
https://news.tvbs.com.tw/politics/1200870 XXX
1831 https://news.cnyes.com/news/id/4351285
1923 https://news.cnyes.com/news/id/4351396
3441 https://news.cnyes.com/news/id/4362649
3533 https://www.businesstoday.com.tw/article/category/80392/post/201911270052/
222 https://news.mingpao.com/ins/%e6%b8%af%e8%81%9e/article/20191204/s00001/1575463599962/%e5%a4%a9%e6%96%87%e5%8f%b0%e7%99%bc%e4%bb%8a%e5%86%ac%e9%a6%96%e5%80%8b%e5%af%92%e5%86%b7%e5%a4%a9%e6%b

1639 https://news.cnyes.com/news/id/4364592
2517 https://www.mirrormedia.mg/story/20191017fin001/
3341 https://m.ctee.com.tw/livenews/aj/chinatimes/20191004001216-260410
4493 https://www.chinatimes.com/newspapers/20191030000495-260204
1640 https://news.cnyes.com/news/id/4366492
3342 https://udn.com/news/story/6811/4069069
4494 https://www.chinatimes.com/newspapers/20190704000346-260204
4904 https://m.ctee.com.tw/livenews/ch/chinatimes/20191023002747-260405
1641 https://www.businesstoday.com.tw/article/category/80392/post/201908220011/
3343 https://www.mirrormedia.mg/story/20191210ent024/
4495 https://www.chinatimes.com/newspapers/20191015000242-260210
1642 https://www.chinatimes.com/newspapers/20191015000370-260206
3344 https://udn.com/news/story/7321/4000958
4496 https://www.mirrormedia.mg/story/20191110web001/
1643 https://news.mingpao.com/ins/%e6%96%87%e6%91%98/article/20191003/s00022/1570020815697/%e7%9b%a3%e8%ad%a6%e6%9c%83%e5%a0%b1%e5%91%8a-%e6%87%89%e4%ba%a4%e7%8d%a8%e7%ab%8b%e8

3200 https://news.cnyes.com/news/id/4378488
3292 https://ol.mingpao.com/php/showbiz3.php?nodeid=1570958280745&subcate=latest&issue=20191013
1591 https://mops.twse.com.tw/mops/web/t05st02?step=1&off=1&firstin=1&TYPEK=sii&i=95&h950=%A5%78%B9%46%B9%71&h951=2308&h952=20191216&h953=171046&h954=%A5%4E%A4%6C%A4%BD%A5%71%A5%78%B9%46%B0%EA%BB%DA%B1%B1%AA%D1%A6%B3%AD%AD%A4%BD%A5%71%A4%BD%A7%69%B8%B3%A8%C6%C5%DC%A7%F3&h955=1&pgname=t05st02
2535 https://www.chinatimes.com/newspapers/20190816000311-260204
3201 https://news.cnyes.com/news/id/4364622
3293 https://www.managertoday.com.tw/books/view/57893
1454 https://m.ctee.com.tw/livenews/ch/chinatimes/20190809002755-260405
1455 https://www.chinatimes.com/realtimenews/20190915001404-260402
1456 https://news.cnyes.com/news/id/4351801
1457 https://udn.com/news/story/7339/4142796
1458 https://m.ctee.com.tw/livenews/aj/chinatimes/20191220003278-260410
1459 https://m.ctee.com.tw/livenews/aj/chinatimes/20191029001786-260410
1460 https://news.cnyes.com/news

...

2747 https://mops.twse.com.tw/mops/web/t05st02?step=1&off=1&firstin=1&TYPEK=otc&i=244&h2440=%A4%B8%AC%B2&h2441=3089&h2442=20191111&h2443=171149&h2444=%A5%4E%A4%6C%A4%BD%A5%71%A4%BD%A7%69%A6%BF%C4%AC%BB%B7%AA%40%A5%FA%B9%71%AA%D1%A5%F7%A6%B3%AD%AD%A4%BD%A5%71%B8%B3%A8%C6%AA%F8%A4%CE%C1%60%B8%67%B2%7A%0D%0A%B2%A7%B0%CA%A8%C6%A9%79&h2445=1&pgname=t05st02
1046 https://www.mirrormedia.mg/story/20191211bus001/
33 https://news.tvbs.com.tw/politics/1191254
https://news.tvbs.com.tw/politics/1191254 XXX
954 https://www.mirrormedia.mg/story/20191226edi001/
2656 http://news.ltn.com.tw/news/society/paper/1301609
2748 https://mops.twse.com.tw/mops/web/t05st02?step=1&off=1&firstin=1&TYPEK=sii&i=180&h1800=%A5%FE%B0%EA%B9%71&h1801=6281&h1802=20190812&h1803=152243&h1804=%A4%BD%A7%69%A5%BB%A4%BD%A5%71%B0%5D%B0%C8%BA%5B%B7%7C%AD%70%A5%44%BA%DE%B2%A7%B0%CA&h1805=1&pgname=t05st02
4570 https://m.ctee.com.tw/livenews/ch/20190724003026-260405
1047 https://m.ctee.com.tw/livenews/gj/chinatimes/20190905004881-260

4177 https://udn.com/news/story/7333/4134885
4269 https://sina.com.hk/news/article/20191107/1/67/1/%E9%9A%B1%E8%97%8F%E5%8A%9F%E8%83%BD-%E7%A9%BF%E7%AA%BF%E7%89%9B%E4%BB%94%E8%A4%B2%E6%9C%89%E5%92%A9%E7%94%A8-%E5%8F%B0%E5%A5%B3%E7%94%9F%E8%A9%B1%E6%8A%93%E7%99%A2%E8%B6%85%E6%96%B9%E4%BE%BF-10810324.html
4705 https://www.chinatimes.com/newspapers/20191003000535-260210
682 http://news.ltn.com.tw/news/society/breakingnews/2891607
866 https://udn.com/news/story/7332/3907379
4178 https://mops.twse.com.tw/mops/web/t05st02?step=1&off=1&firstin=1&TYPEK=otc&i=216&h2160=%A4%54%B6%EA&h2161=4416&h2162=20191129&h2163=170114&h2164=%A4%BD%A7%69%A5%BB%A4%BD%A5%71%B8%B3%A8%C6%B7%7C%B3%71%B9%4C%B2%D7%A4%EE%BB%50%A9%FA%BD%6E%AB%D8%B3%5D%AA%D1%A5%F7%A6%B3%AD%AD%A4%BD%A5%71%A5%48%A6%40%A6%50%A7%EB%0D%0A%B8%EA%BF%B3%AB%D8%A4%E8%A6%A1%A9%D2%C3%B1%AD%71%A4%A7%A6%58%AB%D8%A4%C0%B0%E2%AB%B4%AC%F9%AE%D7%A1%43&h2165=1&pgname=t05st02
4270 https://www.chinatimes.com/realtimenews/20190726002288-260402
683 http://new

4091 http://technews.tw/2019/10/31/notepad-plus-plus-poke-china-glass-heart-but-it-still-does-not-get-github-attention/
4914 https://www.businesstoday.com.tw/article/category/80392/post/201907180021/穿玩偶裝讓她手抖不停、中暑倒地不能拿下頭套…迪士尼「夢幻樂園」的幕後血淚
780 https://m.ctee.com.tw/livenews/ch/chinatimes/20190903003034-260405
2482 https://news.ltn.com.tw/news/society/paper/1322170
2574 https://news.cnyes.com/news/id/4415495
4092 https://udn.com/news/story/6813/3994080
4881 http://finance.technews.tw/2019/07/16/labor-insurance-is-difficult-to-save-the-elderly-annuity-expenditure-in-may-reached-20-9-billion/
4968 https://www.mirrormedia.mg/story/20190930mkt001/
781 https://www.mirrormedia.mg/story/20191224edi017/
2483 https://mops.twse.com.tw/mops/web/t05st02?step=1&off=1&firstin=1&TYPEK=sii&i=131&h1310=%A5%FC%C2%D7%AA%F7&h1311=2886&h1312=20190920&h1313=170318&h1314=%A5%4E%A5%BB%A4%BD%A5%71%A4%A7%A4%6C%A4%BD%A5%71%A5%FC%C2%D7%B0%EA%BB%DA%B0%D3%B7%7E%BB%C8%A6%E6%A4%BD%A7%69%A8%E4%0D%0A%AE%E7%A6%CB%AD%5D%B0%CF

Exception ignored in: <function WeakValueDictionary.__init__.<locals>.remove at 0x0000018EAF313C80>
Traceback (most recent call last):
  File "C:\Users\9Suser\Anaconda3\envs\pyscraper\lib\weakref.py", line 110, in remove
    self = selfref()
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
data.loc[84, 'content']

In [ ]:
data.to_csv(r'E:\Desktop\模型訓練資料\tbrain_train_final_0603_crawl.csv')